## Tools Based Multi Source RAG

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki.name


'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
loader=WebBaseLoader("https://missionshakti.odisha.gov.in/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000159CCEB3EC0>, search_kwargs={})

In [18]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"shakti_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [19]:
retriever_tool.name


'shakti_search'

In [20]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [21]:
tools=[wiki,arxiv,retriever_tool]


In [22]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Development\\Shakti-RAG\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='shakti_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000015941628FE0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vec

In [23]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [24]:
"""from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages"""
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant"),
  ("placeholder", "{chat_history}"),
  ("human", "{input}"),
  ("placeholder", "{agent_scratchpad}"),
])

In [25]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [26]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad', 'chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [30]:
agent_executor.invoke({"input":"Tell me about millet products in Mission Shakti Odisha"})




> Entering new AgentExecutor chain...

Invoking: `shakti_search` with `{'query': 'millet products in Mission Shakti Odisha'}`


Home | Department of Mission Shakti






























































          Sorry, you need to enable JavaScript to visit this website.
        



      Skip to main content
    
















Government of Odisha
ଓଡ଼ିଶା ସରକାର 




Login
Register








 
 





 A+ 
 A  
 A- 
 A 
 A 






















Search

















EnglishOdia












 Department of Mission Shakti  
Government of Odisha












Shri Mohan Charan Majhi 
Hon'ble Chief Minister

Shri Mohan Charan Majhi 
Hon'ble Chief Minister





















HomeAbout UsOverviewVision & MissionCommunity StructureOrganisational StructureWho's whoAdministrationState Project Monitoring UnitNotificationRules & GuidelinesLetter & CircularPublicationReportMagazinesNewslettersPostersLeafletsBooksProgrammeInstitution BuildingPartnershipORMASRSETIUNFPAUNICE

{'input': 'Tell me about millet products in Mission Shakti Odisha',
 'output': "I couldn't find specific information about millet products in Mission Shakti Odisha. If you have any other questions or need assistance with something else, feel free to ask!"}

In [17]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})




> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.'}

In [19]:
agent_executor.invoke({"input":"Tell me about Mission Shakti Odisha Heirarchy"})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Mission Shakti Odisha Hierarchy'}`


Page: List of schemes of the government of India
Summary: The Government of India has social welfare and social security schemes for India's citizens funded either by the central government, state govI couldn't find specific information about the hierarchy of Mission Shakti Odisha on Wikipedia. Would you like me to search for more information using a different tool?

> Finished chain.


{'input': 'Tell me about Mission Shakti Odisha Heirarchy',
 'output': "I couldn't find specific information about the hierarchy of Mission Shakti Odisha on Wikipedia. Would you like me to search for more information using a different tool?"}